# Project 1: Customer Database
**This is the first of three mandatory projects to be handed in as part of the assessment for the course 02807 Computational Tools for Data Science at Technical University of Denmark, autumn 2019.**

#### Practical info
- **The project is to be done in groups of at most 3 students**
- **Each group has to hand in _one_ Jupyter notebook (this notebook) with their solution**
- **The hand-in of the notebook is due 2019-10-13, 23:59 on DTU Inside**

#### Your solution
- **Your solution should be in Python**
- **For each question you may use as many cells for your solution as you like**
- **You should document your solution and explain the choices you've made (for example by using multiple cells and use Markdown to assist the reader of the notebook)**
- **You should not remove the problem statements, and you should not modify the structure of the notebook**
- **Your notebook should be runnable, i.e., clicking [>>] in Jupyter should generate the result that you want to be assessed**
- **You are not expected to use machine learning to solve any of the exercises**
- **You will be assessed according to correctness and readability of your code, choice of solution, choice of tools and libraries, and documentation of your solution**

## Introduction
Your team has been hired by the company X as data scientists. X makes gadgets for a wide range of industrial and commercial clients.

As in-house data scientists, your teams first task, as per request from your new boss, is to optimize business operations. You have decided that a good first step would be to analyze the companys historical sales data to gain a better understanding of where profit is coming from. It may also reveal some low hanging fruit in terms of business opportunities.

To get started, you have called the IT department to get access to the customer and sales transactions database. To your horror you've been told that such a database doens't exist, and the only record of sales transactions is kept by John from finance in an Excel spreadsheet. So you've emailed John asking for a CSV dump of the spreadsheet...

In this project you need to clean the data you got from John, enrich it with further data, prepare a database for the data, and do some data analysis. The project is comprised of five parts. They are intended to be solved in the order they appear, but it is highly recommended that you read through all of them and devise an overall strategy before you start implementing anything.

## Part 1: Cleaning the data
John has emailed you the following link to the CSV dump you requested.

- [transactions.csv](https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv)

It seems as though he has been a bit sloppy when keeping the records. 

In this part you should:
- Explain what the data is
- Clean it to prepare it for inserting into a database and doing data analysis 

---
### Solution

In [1]:
import pandas as pd
import numpy as np
import re
import pickle

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/patrickcording/02807-comp-tools/master/docker/work/data/transactions.csv",encoding="utf-8")

In [3]:
df.head()

,part,company,country,city,price,date
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05
1,60505-2867,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26
2,24385-268,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30
3,76117-001,Twitterbeat,France,Annecy,1075.82€,2016-01-02 02:32:30
4,44946-1046,Chatterbridge,Spain,Barcelona,412.55€,2016-01-02 04:51:55


In [4]:
df.describe()

,part,company,country,city,price,date
count,20558,20568,18397,20535,20567,20568
unique,100,35,13,30,19214,20552
top,17156-617,Thoughtmix,Portugal,Amadora\t,-,10/04/2017
freq,300,2795,7383,2787,5,7


In [5]:
df.dtypes

part       object
company    object
country    object
city       object
price      object
date       object
dtype: object

In [6]:
for column in df.columns:
    print("Data type in {} is {}.".format(column,type(df[column][0])))

Data type in part is <class 'str'>.
Data type in company is <class 'str'>.
Data type in country is <class 'str'>.
Data type in city is <class 'str'>.
Data type in price is <class 'str'>.
Data type in date is <class 'str'>.


In [7]:
# check for empty string
for column in df.columns:
    print("" in df[column])

False
False
False
False
False
False


In [8]:
# check for NaN
for column in df.columns:
    print("column {} has {} null value".format(column, df[column].isnull().sum().sum()))

column part has 10 null value
column company has 0 null value
column country has 2171 null value
column city has 33 null value
column price has 1 null value
column date has 0 null value


### part column

In [9]:
df.part.nunique()

100

In [10]:
df.part.value_counts(dropna = False).head()

17156-617     300
37205-992     295
0268-6107     293
52959-433     293
54868-0823    292
Name: part, dtype: int64

#### handle missing values in `part`

In [11]:
df[df.part.isnull()]

,part,company,country,city,price,date
14916,NaN,Yozio,Greece,Patras,518.38€,2018-02-17 21:43:43
14917,NaN,Brainsphere,Portugal,Braga,957.24€,2018-02-17 22:12:24
14918,NaN,Lajo,Greece,Thessaloniki,966.06€,2018-02-17 22:54:49
14919,NaN,Roodel,Portugal,Aranhas,873.65€,2018-02-17 23:36:52
17524,NaN,Yozio,Greece,Patras,627.32€,2018-07-12 03:28:46
17525,NaN,Thoughtmix,Portugal,Amadora\t,825.8€,2018-07-12 05:34:07
17526,NaN,Gabcube,Portugal,Almada,188.31€,2018-07-12 06:49:44
17527,NaN,Buzzbean,Germany,Düsseldorf,429.67€,2018-07-12 07:03:50
17528,NaN,Zoonder,United States,Boston,$521.72,2018-07-12 08:38:56
17529,NaN,Twitterbeat,France,Annecy,1031.46€,2018-07-12 09:48:17


In [12]:
prop_nan_part = len(df[df.part.isnull()])/len(df)
prop_nan_part

0.0004861921431349669

Since the proportion of missing values in the part column is only 0.049% of the entire dataset, we can remove the rows with missing part entries.

In [13]:
df=df.dropna(subset=['part'])

In [14]:
df[df.part.isnull()]

,part,company,country,city,price,date


### company column

#### clean strings in `company`

In [15]:
df.company.unique()

array(['Chatterbridge', 'Lajo', 'Flipstorm', 'Twitterbeat', 'Voomm',
       'Buzzbean', 'Zooxo', 'Brainsphere', 'Thoughtmix', 'Wordify',
       'Teklist', 'Avaveo', 'Ntags', 'Innojam', 'Shufflebeat', 'Zoonder',
       'Kanoodle', 'Gabcube', 'Roodel', 'Riffpath', 'Eimbee', 'Yozio',
       'Rhycero', 'Realpoint', 'Gabtune', 'Bubblemix', 'Gevee', 'Tagtune',
       'Zooxo.', 'Laj0', 'Ntagz', ' -', ' a', 'aa', 'Thoughtmixz'],
      dtype=object)

In [16]:
df.company.value_counts(dropna=False)

Thoughtmix       2794
Twitterbeat      2267
Zooxo            1670
Chatterbridge    1589
Shufflebeat      1553
Ntags            1514
Buzzbean         1253
Brainsphere      1241
Flipstorm        1193
Wordify           968
Yozio             653
Roodel            626
Eimbee            498
Zoonder           453
Teklist           428
Gabcube           356
Voomm             250
Lajo              219
Avaveo            212
Rhycero           204
Realpoint         158
Riffpath          151
Kanoodle          127
Bubblemix          54
Innojam            44
Gevee              36
Gabtune            27
Tagtune            12
Zooxo.              2
 a                  1
Thoughtmixz         1
 -                  1
Laj0                1
Ntagz               1
aa                  1
Name: company, dtype: int64

From the above result, it can be seen that some erroneous company names/poorly formatted company names. These include:
* ` a`,`aa`,` -` each has occurred once in the dataset. They are likely to be the erroneous entries. Simply remove them.
* `Laj0` should be standardized to `Lajo`. Likewise, `Zooxo.` should be standardized to `Zooxo`.

In [17]:
df.company = df.company.replace([' a','aa',' -'], '')
df=df[df.company != '']

In [18]:
df.company = df.company.replace('Zooxo.','Zooxo')
df.company = df.company.replace('Laj0','Lajo')

In [19]:
df.company.value_counts(dropna=False)

Thoughtmix       2794
Twitterbeat      2267
Zooxo            1672
Chatterbridge    1589
Shufflebeat      1553
Ntags            1514
Buzzbean         1253
Brainsphere      1241
Flipstorm        1193
Wordify           968
Yozio             653
Roodel            626
Eimbee            498
Zoonder           453
Teklist           428
Gabcube           356
Voomm             250
Lajo              220
Avaveo            212
Rhycero           204
Realpoint         158
Riffpath          151
Kanoodle          127
Bubblemix          54
Innojam            44
Gevee              36
Gabtune            27
Tagtune            12
Thoughtmixz         1
Ntagz               1
Name: company, dtype: int64

In [20]:
df[df['company'].isnull()]

,part,company,country,city,price,date


### country column

#### clean strings in `country`

In [21]:
df.country.unique()

array(['Spain', 'Greece', 'France', 'Germany', 'United Kingdom',
       'Portugal', 'United States', 'Netherlands', 'Japan', 'Switzerland',
       nan, 'US', 'Tyskland', 'Portuga'], dtype=object)

From the `unique()` result, we can see that `United States` and `US` are the same and thus should be standardized. 
<br>
Similarly, `Portugal` and `Portuga` should be standardized. 
<br>
Additionally, `Tyskland` means `Germany` in Swedish. Thus they need to be standardized too.

In [22]:
df.country = df.country.replace('US', 'United States')
df.country = df.country.replace('Portuga', 'Portugal')
df.country = df.country.replace('Tyskland', 'Germany')

In [23]:
df.country.unique()

array(['Spain', 'Greece', 'France', 'Germany', 'United Kingdom',
       'Portugal', 'United States', 'Netherlands', 'Japan', 'Switzerland',
       nan], dtype=object)

#### handle missing values in  `country` 

In [24]:
df[df.country.isnull()].head()

,part,company,country,city,price,date
2528,52380-1102,Teklist,NaN,Arnhem,357.78,2016-04-21 04:07:31
2947,52125-136,Brainsphere,NaN,Braga,493.94€,2016-05-10 11:13:43
2956,16714-295,Teklist,NaN,Arnhem,624.4€,2016-05-10 21:57:15
2994,76335-006,Buzzbean,NaN,Düsseldorf,355.24€,2016-05-12 15:17:37
3948,54473-578,Brainsphere,NaN,Braga,123.32€,2016-07-01 01:09:40


In [25]:
len(df[df.country.isnull()])

2171

In [26]:
prop_nan_country = len(df[df.country.isnull()])/len(df)
prop_nan_country

0.1056190707856969

Since the fraction of missing values in the country column is 10.43% of the entire dataset which is considerably large, we cannot simply remove the rows with missing countries entries as doing so will result in much information being lost.
<br>
<br>
Instead, we can deduce some of the country names from their corresponding cities.

In [27]:
df.groupby(['city'])['country'].value_counts(dropna = False).head()

city       country 
Almada     Portugal     323
           NaN           33
Amadora\t  Portugal    2511
           NaN          275
Amiens     France       452
Name: country, dtype: int64

In [28]:
city_country_mapping = df.groupby(['city'])['country'].apply(lambda grp: list(grp.value_counts().index)[0]).to_dict()
city_country_mapping

{'Almada': 'Portugal',
 'Amadora\t': 'Portugal',
 'Amiens': 'France',
 'Amsterdam': 'Netherlands',
 'Annecy': 'France',
 'Aranhas': 'Portugal',
 'Arcueil': 'France',
 'Arnhem': 'Netherlands',
 'Asaka': 'Japan',
 'Athens': 'Greece',
 'Barcelona': 'Spain',
 'Boston': 'United States',
 'Braga': 'Portugal',
 'Champagnole': 'France',
 'Düsseldorf': 'Germany',
 'Heraklion': 'Greece',
 'Lisbon': 'Portugal',
 'London': 'United Kingdom',
 'Lyon': 'France',
 'Monção': 'Portugal',
 'Nanterre': 'France',
 'New York': 'United States',
 'Nice': 'France',
 'Niihama': 'Japan',
 'Paris': 'France',
 'Patras': 'Greece',
 'Porto': 'Portugal',
 'Thessaloniki': 'Greece',
 'Vila Fria': 'Portugal',
 'Zürich': 'Switzerland'}

In [29]:
df[df.country.isnull()].index

Int64Index([ 2528,  2947,  2956,  2994,  3948,  3949,  3950,  3951,  3952,
             3953,
            ...
            12843, 12978, 13721, 14438, 16146, 16147, 16148, 16149, 16150,
            16151],
           dtype='int64', length=2171)

In [30]:
def impute_countries(row):
    try:
        res=city_country_mapping[row.city]
    except KeyError:
        res=float('NaN')
    return res

In [31]:
df.country = df.apply(impute_countries, axis = 1)

In [32]:
len(df[df.country.isna()])

33

In [33]:
df[df.country.isna()]

,part,company,country,city,price,date
3136,68462-565,Brainsphere,NaN,NaN,1057.85€,2016-05-20 08:59:53
3137,54092-515,Shufflebeat,NaN,NaN,772.92€,2016-05-20 09:55:24
3138,0185-0373,Brainsphere,NaN,NaN,1012.8€,2016-05-20 10:42:10
12820,50563-113,Ntags,NaN,NaN,507.49€,2017-10-20 04:03:11
12821,55154-5057,Brainsphere,NaN,NaN,421.64€,2017-10-20 13:23:34
12822,36987-1697,Ntags,NaN,NaN,937.08€,2017-10-20 18:25:00
12823,68462-565,Thoughtmix,NaN,NaN,777.58€,2017-10-20 21:21:18
12824,0603-6134,Yozio,NaN,NaN,775.05€,2017-10-20 23:17:59
12825,58118-5060,Yozio,NaN,NaN,157.53€,2017-10-20 23:49:53
12826,51060-032,Thoughtmix,NaN,NaN,176.36€,2017-10-21 00:11:55


### city column

#### clean strings in `city`

In [34]:
df.city.unique()

array(['Barcelona', 'Thessaloniki', 'Athens', 'Annecy', 'Paris',
       'Düsseldorf', 'London', 'Braga', 'Nanterre', 'Amadora\t',
       'New York', 'Arnhem', 'Nice', 'Lisbon', 'Amsterdam', 'Porto',
       'Boston', 'Niihama', 'Almada', 'Aranhas', 'Heraklion', 'Amiens',
       'Patras', 'Arcueil', 'Lyon', 'Asaka', 'Champagnole', 'Zürich', nan,
       'Monção', 'Vila Fria'], dtype=object)

Result from `unique()` shows that there is an additional `\t` in the city `Amadora`. Remove the`\t`s:

In [35]:
df.city = df.city.str.replace("\t","")

In [36]:
df.city.unique()

array(['Barcelona', 'Thessaloniki', 'Athens', 'Annecy', 'Paris',
       'Düsseldorf', 'London', 'Braga', 'Nanterre', 'Amadora', 'New York',
       'Arnhem', 'Nice', 'Lisbon', 'Amsterdam', 'Porto', 'Boston',
       'Niihama', 'Almada', 'Aranhas', 'Heraklion', 'Amiens', 'Patras',
       'Arcueil', 'Lyon', 'Asaka', 'Champagnole', 'Zürich', nan, 'Monção',
       'Vila Fria'], dtype=object)

#### handle missing values in `city`

In [37]:
df[df.city.isnull()]

,part,company,country,city,price,date
3136,68462-565,Brainsphere,NaN,NaN,1057.85€,2016-05-20 08:59:53
3137,54092-515,Shufflebeat,NaN,NaN,772.92€,2016-05-20 09:55:24
3138,0185-0373,Brainsphere,NaN,NaN,1012.8€,2016-05-20 10:42:10
12820,50563-113,Ntags,NaN,NaN,507.49€,2017-10-20 04:03:11
12821,55154-5057,Brainsphere,NaN,NaN,421.64€,2017-10-20 13:23:34
12822,36987-1697,Ntags,NaN,NaN,937.08€,2017-10-20 18:25:00
12823,68462-565,Thoughtmix,NaN,NaN,777.58€,2017-10-20 21:21:18
12824,0603-6134,Yozio,NaN,NaN,775.05€,2017-10-20 23:17:59
12825,58118-5060,Yozio,NaN,NaN,157.53€,2017-10-20 23:49:53
12826,51060-032,Thoughtmix,NaN,NaN,176.36€,2017-10-21 00:11:55


In [38]:
len(df[df.city.isnull()])

33

In [39]:
df.groupby(["company","country",'city']).count()

part  price  date
company       country        city                           
Avaveo        France         Nice           212    212   212
Brainsphere   Portugal       Braga         1236   1236  1236
                             Monção           1      1     1
Bubblemix     Japan          Asaka           54     54    54
Buzzbean      Germany        Düsseldorf    1253   1253  1253
Chatterbridge Spain          Barcelona     1589   1589  1589
Eimbee        France         Amiens         498    498   498
Flipstorm     France         Nanterre       381    381   381
              Greece         Athens         812    812   812
Gabcube       Portugal       Almada         356    356   356
Gabtune       France         Lyon            27     27    27
Gevee         France         Champagnole     36     36    36
Innojam       Netherlands    Amsterdam       44     44    44
Kanoodle      Japan          Niihama        126    126   126
Lajo          Greece         Thessaloniki   220    220   220
Ntags         Portugal       Lisbon        1508   1508  1508
Ntagz         Portugal       Lisbon           1      1     1
Realpoint     Portugal       Lisbon         158    158   158
Rhycero       France         Arcueil        204    204   204
Riffpath      Greece         Heraklion      151    151   151
Roodel        Portugal       Aranhas        626    625   626
Shufflebeat   Portugal       Porto         1550   1550  1550
Tagtune       Switzerland    Zürich          12     12    12
Teklist       Netherlands    Arnhem         426    426   426
Thoughtmix    Portugal       Amadora       2785   2785  2785
                             Vila Fria        1      1     1
Thoughtmixz   Portugal       Amadora          1      1     1
Twitterbeat   France         Annecy        2266   2266  2266
Voomm         France         Paris          250    250   250
Wordify       United States  New York       966    966   966
Yozio         Greece         Patras         651    651   651
Zoonder       United States  Boston         452    452   452
Zooxo         United Kingdom London        1669   1669  1669

In [40]:
prop_nan_city = len(df[df.city.isnull()])/len(df)
prop_nan_city

0.001605448795913403

Since the proportion of missing values in the city column is only 0.16% of the entire dataset, we can remove the rows with missing city entries.
<br>
<br>
Moreover, the table above shows that all but three records have missing values in both country and city. This implies that dropping records that have missing values in the city column does not cause a big loss in information in the country column, since only three such country records are not null.

In [41]:
df=df.dropna(subset=['city'])

In [42]:
df[df.city.isnull()]

,part,company,country,city,price,date


### date column

In [43]:
# string to datetime object
df['parsed_date'] = pd.to_datetime(df['date'],errors='coerce', infer_datetime_format=True)

In [44]:
# checking
df[df.parsed_date.isnull()]

,part,company,country,city,price,date,parsed_date
3539,17156-617,Thoughtmix,Portugal,Amadora,791.86€,2016-06-32 07:22:28,NaT
3540,54868-5165,Shufflebeat,Portugal,Porto,525.24€,2016-06-32 08:08:48,NaT


In [45]:
df = df.dropna()

### price column

In [46]:
# original intention was to find all currency signs present
# but it turned out that there are other poorly formatted cells other than currency signs
res=[]
for p in df.price:
    for c in p:
        if c.isdigit() == False:
            res.append(c)
x=np.array(res)
np.unique(x)

array(['$', '-', '.', 'a', 'd', 'i', 'n', 'o', 'v', '£', '¥', '€'],
      dtype='<U1')

In [47]:
# find what are those cells
res=[]
for p in df.price:
    for c in p:
        if c in ['a', 'd', 'i', 'n', 'o', 'v']:
            print(p)
            break

void
void
na
na
na


In [48]:
print(len(df))
df=df[df.price != 'void']
df=df[df.price != 'na']
print(len(df))

20519
20514


In [49]:
def parse_price(string):
    amount=re.match(r'([^-\d\.]?)(-?\d*\.?\d*)([\D]?)', str(string))
    lst=[i for i in amount.groups() if i != '']
#     if len(lst) != 2:
#         return float('NaN')
    try:
        res=float(lst[0])
    except ValueError:
        res=float(lst[1])
    return res

In [50]:
def parse_symbol(string):
    amount=re.match(r'([^-\d\.]?)(-?\d*\.?\d*)([\D]?)', str(string))
    lst=[i for i in amount.groups() if i != '']
#     if len(lst) != 2:
#         return float('NaN')
    try:
        number=float(lst[0])
        sign=lst[1]
    except ValueError:
        sign=lst[0]
    return sign

In [51]:
def find_currency(string):
    currency=re.findall("[€$¥£]",string)
    if currency:
        symbol=currency[0]
        if symbol == "€":
            return "EUR"
        elif symbol == "$":
            return "USD"
        elif symbol == "¥":
            return "CNY"
        elif symbol == "£":
            return "GBP"
    else:
        print("error: can't find currenct symbol in {}".format(string))

In [52]:
df["currency"]=df.price.apply(find_currency)

error: can't find currenct symbol in 465.6
error: can't find currenct symbol in 1266.68
error: can't find currenct symbol in 829.3
error: can't find currenct symbol in 357.78
error: can't find currenct symbol in -
error: can't find currenct symbol in -
error: can't find currenct symbol in -
error: can't find currenct symbol in -
error: can't find currenct symbol in -


In [53]:
df=df[df['price'] != "465.6"]
df=df[df['price'] != "1266.68"]
df=df[df['price'] != "829.3"]
df=df[df['price'] != "357.78"]
df=df[df['price'] != "-"]

In [54]:
df['price_amount']=df.price.apply(parse_price)

In [55]:
df['price_symbol']=df.price.apply(parse_symbol)

In [56]:
df.tail()

,part,company,country,city,price,date,parsed_date,currency,price_amount,price_symbol
20563,21695-267,Roodel,Portugal,Aranhas,606.37€,2018-12-31 20:48:14,2018-12-31 20:48:14,EUR,606.37,€
20564,49999-737,Thoughtmix,Portugal,Amadora,828.37€,2018-12-31 21:24:17,2018-12-31 21:24:17,EUR,828.37,€
20565,52343-025,Brainsphere,Portugal,Braga,686.1€,2018-12-31 21:38:02,2018-12-31 21:38:02,EUR,686.10,€
20566,49288-0285,Flipstorm,Greece,Athens,1051.08€,2019-02-21 20:05:00,2019-02-21 20:05:00,EUR,1051.08,€
20567,43419-018,Flipstorm,Greece,Athens,139.56€,2019-05-14 22:48:39,2019-05-14 22:48:39,EUR,139.56,€


In [57]:
df.isnull().sum().sum()

0

In [58]:
df=df.drop(columns='date')
df.rename(columns={"parsed_date" : "date", 
                   "currency": "currency_name",
                   "price_symbol":"currency_symbol"}, inplace=True)

In [59]:
df.head()

,part,company,country,city,price,date,currency_name,price_amount,currency_symbol
0,54868-5165,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05,EUR,784.79,€
1,60505-2867,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26,EUR,187.99,€
2,24385-268,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30,EUR,221.73,€
3,76117-001,Twitterbeat,France,Annecy,1075.82€,2016-01-02 02:32:30,EUR,1075.82,€
4,44946-1046,Chatterbridge,Spain,Barcelona,412.55€,2016-01-02 04:51:55,EUR,412.55,€


---
## Part 2: Enriching the data

A common task for a data scientists is to combine or enrich data from internal sources with data available from external sources. The purpose of this can be either to fix issues with the data or to make it easier to derive insights from the data.

In this part you should enrich your data with data from at least one external source. You may look to part 4 for some  inspiration as to what is required. Your solution should be automated, i.e., you can not ask the reader of your notebook to download any data manually. You should argue why and what you expect to achieve by the enrichments you are doing.

---
### Solution

In [60]:
import datetime
from forex_python.converter import CurrencyRates

In [61]:
c = CurrencyRates()

In [62]:
def ToUSD(currency,date):
    return c.convert(currency, 'USD', 1 , date)

In [63]:
USD=[]
dic={}
for i in range(len(df)):
    if i == 2000:
        print("I'm 10% done!")
    if i == 5000:
        print("Hang on, Ms.Pretty/Mr.Handsome. I'm doing my best!")
    if i == 10000:
        print("Yeah, I'm 50% done!")
    if i == 15000:
        print("Waiting is boring, why not sing a song? La la la ~")
    if i == 20000:
        print("Almost there, give me 10s!")
    date=df.iloc[i,5].date()
    currency=df.iloc[i,6]
    key=(currency,date)
    if key in dic:
        USD.append(dic[key])
    else:
        dic[key] = ToUSD(currency,date)
        USD.append(dic[key])
print("YAS! I'm done!")

I am 10% done!
Hang on, Ms.Pretty/Mr.Handsome. I am doing my best!
Yeah, I am 50% done!
Waiting is boring, why not sing a song? La la la ~
Almost there, give me 10s!
yes, I am done


In [64]:
df['exchange_rate_to_USD'] = USD

In [65]:
df['standardized_USD'] = df.price_amount * df.exchange_rate_to_USD

In [66]:
df.tail()

,part,company,country,city,price,date,currency_name,price_amount,currency_symbol,exchange_rate_to_USD,standardized_USD
20563,21695-267,Roodel,Portugal,Aranhas,606.37€,2018-12-31 20:48:14,EUR,606.37,€,1.1450,694.293650
20564,49999-737,Thoughtmix,Portugal,Amadora,828.37€,2018-12-31 21:24:17,EUR,828.37,€,1.1450,948.483650
20565,52343-025,Brainsphere,Portugal,Braga,686.1€,2018-12-31 21:38:02,EUR,686.10,€,1.1450,785.584500
20566,49288-0285,Flipstorm,Greece,Athens,1051.08€,2019-02-21 20:05:00,EUR,1051.08,€,1.1354,1193.396232
20567,43419-018,Flipstorm,Greece,Athens,139.56€,2019-05-14 22:48:39,EUR,139.56,€,1.1226,156.670056


save the above dataframe as a pickle object for easy access:

In [71]:
with open(r"df.pickle", "wb") as output_file:
    pickle.dump(df, output_file)

In [67]:
# import requests
# import json

# key="29a4fc85a077fabaf3d550608a5c6e3b"
# baseurl="http://www.apilayer.net/api/live?access_key={}&from={}"

# test="https://apilayer.net/api/historical?access_key={}&date=2019-01-05".format(key)

# requests.get(test).json()

# def getRate(currency):
#     url=baseurl.format(key,currency)
#     response=requests.get(url)
#     data=response.json()['quotes']["USD"+currency]
#     return data

# df.currency_name.unique()

# exchange_rate={}
# for currency in df.currency_name.unique().tolist():
#     if currency == "USD":
#         exchange_rate["USD"] = 1
#     else:
#         exchange_rate[currency]=getRate(currency)

# exchange_rate



---
## Part 3: Creating a database
Storing data in a relational database has the advantages that it is persistent, fast to query, and it will be easier access for other employees at Weyland-Yutani.

In this part you should:
- Create a database and table(s) for the data
- Insert data into the tables

You may use SQLite locally to do this. You should argue why you choose to store your data the way you do. 

---
### Solution

Since the current dataframe `df` does not have a primary key, we create another column called `key` and fill this column with the index of each row, i.e. use the row index as the primary key 

In [2]:
with open(r"df.pickle", "rb") as output_file:
    df = pickle.load(output_file)

In [4]:
df.insert(0, 'key', range(len(df)))

In [5]:
df

,key,part,company,country,city,price,date,currency_name,price_amount,currency_symbol,exchange_rate_to_USD,standardized_USD
0,0,54868-5165,Chatterbridge,Spain,Barcelona,784.79€,2016-01-02 00:01:05,EUR,784.79,€,1.088700,854.400873
1,1,60505-2867,Lajo,Greece,Thessaloniki,187.99€,2016-01-02 00:05:26,EUR,187.99,€,1.088700,204.664713
2,2,24385-268,Flipstorm,Greece,Athens,221.73€,2016-01-02 00:18:30,EUR,221.73,€,1.088700,241.397451
3,3,76117-001,Twitterbeat,France,Annecy,1075.82€,2016-01-02 02:32:30,EUR,1075.82,€,1.088700,1171.245234
4,4,44946-1046,Chatterbridge,Spain,Barcelona,412.55€,2016-01-02 04:51:55,EUR,412.55,€,1.088700,449.143185
5,5,16729-167,Chatterbridge,Spain,Barcelona,359.52€,2016-01-02 07:20:59,EUR,359.52,€,1.088700,391.409424
6,6,52125-444,Voomm,France,Paris,266.62€,2016-01-02 07:40:37,EUR,266.62,€,1.088700,290.269194
7,7,43419-018,Buzzbean,Germany,Düsseldorf,103.45€,2016-01-02 08:57:57,EUR,103.45,€,1.088700,112.626015
8,8,54092-515,Zooxo,United Kingdom,London,£704.94,2016-01-02 09:09:01,GBP,704.94,£,1.483344,1045.668204
9,9,24286-1562,Lajo,Greece,Thessaloniki,317.65€,2016-01-02 11:01:32,EUR,317.65,€,1.088700,345.825555


In [6]:
import sqlite3

In [14]:
conn = sqlite3.connect('project1.sqlite')
c = conn.cursor()

In [15]:
df.columns

Index(['key', 'part', 'company', 'country', 'city', 'price', 'date',
       'currency_name', 'price_amount', 'currency_symbol',
       'exchange_rate_to_USD', 'standardized_USD'],
      dtype='object')

In [21]:
c.execute('''DROP TABLE IF EXISTS customer;''')
conn.commit()

In [22]:
c.execute('''CREATE TABLE IF NOT EXISTS customer(
key INTEGER PRIMARY KEY, 
part varchar(20), 
company varchar(20), 
country varchar(30), 
city varchar(20), 
price varchar(10), 
date DATETIME, 
currency_name varchar(3), 
price_amount float(10), 
currency_symbol varchar(1), 
exchange_rate_to_USD float(10), 
standardized_USD float(10))''')
conn.commit()

In [23]:
df.to_sql('customer', conn, if_exists='replace', index=False)
conn.commit()

In [28]:
# code to test whether the sqlite table created is the same as the original pandas dataframe

c.execute('SELECT Count(*) FROM customer')
print('number of rows in the sqlite table is {}'.format(c.fetchone()[0]))
print('number of rows in the pandas dataframe is {}'.format(len(df)))
conn.commit()


number of rows in the sqlite table is 20505
number of rows in the pandas dataframe is 20505


In [78]:
# code to test whether the sqlite table created is the same as the original pandas dataframe
'''
c.execute('SELECT * FROM customer')
print(c.fetchone())
print(c.fetchmany(100))
conn.commit()
'''

(0, '54868-5165', 'Chatterbridge', 'Spain', 'Barcelona', '784.79€', '2016-01-02 00:01:05', 'EUR', 784.79, '€', 1.0887, 854.4008729999999)
[(1, '60505-2867', 'Lajo', 'Greece', 'Thessaloniki', '187.99€', '2016-01-02 00:05:26', 'EUR', 187.99, '€', 1.0887, 204.664713), (2, '24385-268', 'Flipstorm', 'Greece', 'Athens', '221.73€', '2016-01-02 00:18:30', 'EUR', 221.73, '€', 1.0887, 241.397451), (3, '76117-001', 'Twitterbeat', 'France', 'Annecy', '1075.82€', '2016-01-02 02:32:30', 'EUR', 1075.82, '€', 1.0887, 1171.245234), (4, '44946-1046', 'Chatterbridge', 'Spain', 'Barcelona', '412.55€', '2016-01-02 04:51:55', 'EUR', 412.55, '€', 1.0887, 449.143185), (5, '16729-167', 'Chatterbridge', 'Spain', 'Barcelona', '359.52€', '2016-01-02 07:20:59', 'EUR', 359.52, '€', 1.0887, 391.409424), (6, '52125-444', 'Voomm', 'France', 'Paris', '266.62€', '2016-01-02 07:40:37', 'EUR', 266.62, '€', 1.0887, 290.269194), (7, '43419-018', 'Buzzbean', 'Germany', 'Düsseldorf', '103.45€', '2016-01-02 08:57:57', 'EUR', 1

In [ ]:
GDP=pd.read_csv("")

---
## Part 4: Analyzing the data
You are now ready to analyze the data. Your goal is to gain some actionable business insights to present to your boss. 

In this part, you should ask some questions and try to answer them based on the data. You should write SQL queries to retrieve the data. For each question, you should state why it is relevant and what you expect to find.

To get you started, you should prepare answers to the following questions. You should add more questions.
#### Who are the most profitable clients?
Knowing which clients that generate the most revenue for the company will assist your boss in distributing customer service ressources.

#### Are there any clients for which profit is declining?
Declining profit from a specific client may indicate that the client is disatisfied with the product. Gaining a new client is often much more work than retaining one. Early warnings about declining profit may help your boss fighting customer churn.


Remember, you are taking this to your new boss, so think about how you present the data.

---
### Solution

---
## Part 5: Performance
Your boss is very impressed with what you have achieved in less than two weeks, and he would like to take your idea of storing the customer and sales data in a relational database to production. However, John is concerned that the solution will not scale. His experience is telling him that you will see many occurrences of the following queries.

- Show all sales to company X between time $t_1$ and time $t_2$
- Show the latest X sales in the database
- Show total sales per company per day

Show that Johns concern is not justified.

---
### Solution

---